In [33]:
import pandas as pd
import numpy as np
import pickle 
import os

### Necesitamos transformar los datos de tal forma que Power BI lo pueda entener y extraer

In [34]:
# Cargar el objeto data_clean desde el archivo pickle
with open('data\\data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

print("Objeto cargado correctamente")

print(loaded_data['Dioxido_azufre']['CDMX_CCA2023'].info())

Objeto cargado correctamente
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6432 entries, 0 to 6431
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Fecha                     6432 non-null   datetime64[ns]
 1   Parámetro                 6432 non-null   object        
 2   Concentraciones horarias  6432 non-null   float64       
 3   Unidad                    6432 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 201.1+ KB
None


#### Para ello se puede crear un dataframe con todas las columnas con la información importante

### Creamos un solo dataframe que contenga toda la información de cada contaminante y estación 

#### Agregamos las columnas de contaminante y estación 

In [35]:
# Convertir diccionario a DataFrame estructurado
df = loaded_data.copy()
lista_data = []
for contaminante, estaciones in loaded_data.items():
    for estacion, df in estaciones.items():
        df = df.copy()
        df['Contaminante'] = contaminante
        df['Estación'] = estacion
        lista_data.append(df)

df_final = pd.concat(lista_data).reset_index(drop=True)
df_final = df_final.rename(columns={'Parámetro': 'Parametro'})
df_final


,Fecha,Parametro,Concentraciones horarias,Unidad,Contaminante,Estación
0,2023-01-01 00:00:00,SO2,0.003,ppm,Dioxido_azufre,CDMX_CCA2023
1,2023-01-01 01:00:00,SO2,0.003,ppm,Dioxido_azufre,CDMX_CCA2023
2,2023-01-01 02:00:00,SO2,0.003,ppm,Dioxido_azufre,CDMX_CCA2023
3,2023-01-01 03:00:00,SO2,0.003,ppm,Dioxido_azufre,CDMX_CCA2023
4,2023-01-01 04:00:00,SO2,0.002,ppm,Dioxido_azufre,CDMX_CCA2023
...,...,...,...,...,...,...
229527,2025-01-01 19:00:00,PM2.5,11.000,µg/m³,PM25,NL_universidad2024
229528,2025-01-01 20:00:00,PM2.5,6.000,µg/m³,PM25,NL_universidad2024
229529,2025-01-01 21:00:00,PM2.5,8.000,µg/m³,PM25,NL_universidad2024
229530,2025-01-01 22:00:00,PM2.5,7.000,µg/m³,PM25,NL_universidad2024


### Cambiemos los nombres a unos más formales y llamativos

In [36]:
df_final['Estación'] = df_final['Estación'].replace(
    {'CDMX_CCA2023': 'CDMX, Centro de Ciencias Atmosféricas'}
)
df_final['Estación'] = df_final['Estación'].replace(
    {'CDMX_UAMI2023': 'CDMX, UAMI'}
)
df_final['Estación'] = df_final['Estación'].replace(
    {'NL_Cadereyta2024': 'NL, Cadereyta'}
)

df_final['Estación'] = df_final['Estación'].replace(
    {'NL_Juarez2024': 'NL, Juarez'}
    
)

df_final['Estación'] = df_final['Estación'].replace(
    {'NL_universidad2024': 'NL, Universidad'}
)


In [37]:
df_final

,Fecha,Parametro,Concentraciones horarias,Unidad,Contaminante,Estación
0,2023-01-01 00:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas"
1,2023-01-01 01:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas"
2,2023-01-01 02:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas"
3,2023-01-01 03:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas"
4,2023-01-01 04:00:00,SO2,0.002,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas"
...,...,...,...,...,...,...
229527,2025-01-01 19:00:00,PM2.5,11.000,µg/m³,PM25,"NL, Universidad"
229528,2025-01-01 20:00:00,PM2.5,6.000,µg/m³,PM25,"NL, Universidad"
229529,2025-01-01 21:00:00,PM2.5,8.000,µg/m³,PM25,"NL, Universidad"
229530,2025-01-01 22:00:00,PM2.5,7.000,µg/m³,PM25,"NL, Universidad"


### Ahora agreguemos las columnas de latitud y longitud de las estaciones

In [38]:
# Diccionario con nombres de estaciones corregidos y coordenadas
estaciones_info = {
    'CDMX_CCA2023': ('CDMX, Centro de Ciencias Atmosféricas', 19.3266, -99.1760),
    'CDMX_UAMI2023': ('CDMX, UAMI', 19.3273, -99.0821),
    'NL_Cadereyta2024': ('NL, Cadereyta', 25.5850, -99.9907),
    'NL_Juarez': ('NL, Juarez', 25.6496, -100.0810),
    'NL_universidad2024': ('NL, Universidad', 25.7216, -100.3124)
}

# Aplicar los cambios en una sola operación
df_final['Estación'] = df_final['Estación'].map(lambda x: estaciones_info.get(x, (x, None, None))[0])
df_final['Latitud'] = df_final['Estación'].map(lambda x: {v[0]: v[1] for k, v in estaciones_info.items()}.get(x, None))
df_final['Longitud'] = df_final['Estación'].map(lambda x: {v[0]: v[2] for k, v in estaciones_info.items()}.get(x, None))


In [39]:
df_final

,Fecha,Parametro,Concentraciones horarias,Unidad,Contaminante,Estación,Latitud,Longitud
0,2023-01-01 00:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
1,2023-01-01 01:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
2,2023-01-01 02:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
3,2023-01-01 03:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
4,2023-01-01 04:00:00,SO2,0.002,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
...,...,...,...,...,...,...,...,...
229527,2025-01-01 19:00:00,PM2.5,11.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229528,2025-01-01 20:00:00,PM2.5,6.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229529,2025-01-01 21:00:00,PM2.5,8.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229530,2025-01-01 22:00:00,PM2.5,7.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124


### Guardemos los datos

In [40]:
import pickle
# Guardar el objeto data_clean en un archivo pickle
with open('data' + os.sep + 'data_PowerBI.pkl', 'wb') as f:
    pickle.dump(df_final, f)
print("Objeto guardado correctamente")

Objeto guardado correctamente


### Ejemplo de como cargarlo

In [41]:
# Cargar el objeto data_clean desde el archivo pickle
with open('data' + os.sep + 'data_PowerBI.pkl', 'rb') as f:
    data= pickle.load(f)

print("Objeto cargado correctamente")


Objeto cargado correctamente


In [42]:
data

,Fecha,Parametro,Concentraciones horarias,Unidad,Contaminante,Estación,Latitud,Longitud
0,2023-01-01 00:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
1,2023-01-01 01:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
2,2023-01-01 02:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
3,2023-01-01 03:00:00,SO2,0.003,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
4,2023-01-01 04:00:00,SO2,0.002,ppm,Dioxido_azufre,"CDMX, Centro de Ciencias Atmosféricas",19.3266,-99.1760
...,...,...,...,...,...,...,...,...
229527,2025-01-01 19:00:00,PM2.5,11.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229528,2025-01-01 20:00:00,PM2.5,6.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229529,2025-01-01 21:00:00,PM2.5,8.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
229530,2025-01-01 22:00:00,PM2.5,7.000,µg/m³,PM25,"NL, Universidad",25.7216,-100.3124
